In [26]:
import onnxruntime as rt
import pandas as pd
import numpy as np


In [27]:
def load_models(model1_path="model_1.onnx", model2_path="model_2.onnx"):
    """
    Loads the ONNX models and prints their input requirements.
    """
    try:
        # Initialize Inference Sessions
        # Provider 'CPUExecutionProvider' is standard for general testing
        sess_1 = rt.InferenceSession(model1_path, providers=['CPUExecutionProvider'])
        sess_2 = rt.InferenceSession(model2_path, providers=['CPUExecutionProvider'])

        print("--- Model Loading Successful ---")

        for i, sess in enumerate([sess_1, sess_2], 1):
            input_meta = sess.get_inputs()[0]
            output_meta = sess.get_outputs()[0]

            print(f"\nModel {i} Metadata:")
            print(f"  Input Name:  {input_meta.name}")
            print(f"  Input Shape: {input_meta.shape}")
            print(f"  Output Name: {output_meta.name}")

        return sess_1, sess_2

    except Exception as e:
        print(f"Error loading models: {e}")
        return None, None

# Usage example:
# session_bad, session_good = load_rotterdam_models()

# Example of how to run a single prediction:
# dummy_input = np.zeros((1, 315), dtype=np.float32)
# result = session_bad.run(None, {'float_input': dummy_input})

In [28]:
def load_data():
    global X_train, y_train, X_test, y_test
    X_train = pd.read_csv("../data/train.csv")
    X_test = pd.read_csv("../data/test.csv")
    y_train = X_train['checked']
    y_test = X_test['checked']
    X_train.drop(columns=['checked'], errors='ignore', inplace=True)
    X_test.drop(columns=['checked'], errors='ignore', inplace=True)

In [29]:
import pandas as pd
import numpy as np
import onnxruntime as rt
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix

def evaluate_classical_metrics(sess, model_name="Model"):
    """
    Evaluates an ONNX model session using standard ML performance metrics
    against the global X_test and y_test datasets.
    """
    # 1. Access the global test data loaded by load_data()
    global X_test, y_test

    # 2. Get the input name from the ONNX session
    input_name = sess.get_inputs()[0].name

    # 3. Prepare features: ONNX requires float32 and a numpy array
    X_features = X_test.values.astype(np.float32)

    # 4. Run Inference
    # sess.run returns a list of outputs; we take the first one (predictions)
    raw_results = sess.run(None, {input_name: X_features})
    y_pred = raw_results[0].flatten()

    # 5. Calculate Metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Note: AUC-ROC usually requires probabilities.
    # If your model only outputs hard classes (0/1), this will be a rough estimate.
    auc = roc_auc_score(y_test, y_pred)

    print(f"\n--- Classical ML Evaluation: {model_name} ---")
    print(f"Accuracy:  {acc:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"AUC-ROC:   {auc:.4f}")
    print("\nConfusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    return {"accuracy": acc, "f1": f1, "auc": auc}

# --- Execution ---
# Assumes you have already run:
# load_data()
# loaded_model1, loaded_model2 = load_models()


In [30]:
load_data()
loaded_model1, loaded_model2 = load_models()


--- Model Loading Successful ---

Model 1 Metadata:
  Input Name:  float_input
  Input Shape: [None, 317]
  Output Name: output_label

Model 2 Metadata:
  Input Name:  float_input
  Input Shape: [None, 317]
  Output Name: output_label


In [31]:
evaluate_classical_metrics(loaded_model1)


--- Classical ML Evaluation: Model ---
Accuracy:  0.8725
F1-Score:  0.4226
AUC-ROC:   0.6413

Confusion Matrix:
[[21472   627]
 [ 2688  1213]]


{'accuracy': 0.8725,
 'f1': np.float64(0.42257446437902807),
 'auc': np.float64(0.6412867933826109)}

In [32]:
evaluate_classical_metrics(loaded_model2)


--- Classical ML Evaluation: Model ---
Accuracy:  0.8563
F1-Score:  0.2237
AUC-ROC:   0.5605

Confusion Matrix:
[[21727   372]
 [ 3363   538]]


{'accuracy': 0.8563461538461539,
 'f1': np.float64(0.2236541259613386),
 'auc': np.float64(0.5605400073373532)}